**Aeronautics Institute of Technology – ITA**

**Computer Vision – CM-203**

**Professors:** 

Marcos Ricardo Omena de Albuquerque Maximo

Gabriel Adriano de Melo


**Instructions:**

Before submitting your lab, be sure that everything is running correctly (in sequence): first, **restart the kernel** (`Runtime->Restart Runtime` in Colab or `Kernel->Restart` in Jupyter). Then, execute all cells (`Runtime->Run All` in Colab or `Cell->Run All` in Jupyter) and verifies that all cells run without any errors, expecially the automatic grading ones, i.e. the ones with `assert`s.

**Do not delete the answer cells**, i.e. the ones that contains `WRITE YOUR CODE HERE` or `WRITE YOUR ANSWER HERE`, because they contain metadata with the ids of the cells for the grading system. For the same reason, **do not delete the test cells**, i.e. the ones with `assert`s. The autograding system executes all the code sequentially, adding extra tests in the test cells. There is no problem in creating new cells, as long as you do not delete answer or test cells. Moreover, keep your solutions within the reserved spaces.

The notebooks are implemented to be compatible with Google Colab, and they install the dependencies and download the datasets automatically. The commands which start with ! (exclamation mark) are bash commands and can be executed in a Linux terminal.

---

# Laboratório 6 - nanoYOLO

Neste laboratório iremos explorar os principais passos necessários na detecção de objetos, sobretudo de métodos de um único estágio, especificamente do YOLO proposta pelo [Joseph Redmon](https://pjreddie.com/darknet/yolo/), em suas versões iniciais. Algo essencial para vocês entenderem é o processo de tornar saídas aparentemente genéricas, contínuas, de tamanhos não definidos, em estruturas discretas que aproximam o valor genérico. Sobretudo que a implementação deste lab é apenas uma forma, apenas uma realização dentre as muitas possíveis.

Assim, o nosso primeiro passo será transformar uma lista de *bounding boxes* em um tensor de tamanho bem definido, que é justamente a saída esperada da rede, o $y$ a ser comparado com o $\hat{y}$ da rede. O segundo passo realizar um transfer learning a partir da arquitetura de uma rede de classificação de objetos, iso é, iremos modificar as últimas camadas de sua arquitetura para termos a estrutura desejada. Em seguida iremos definir a função custo que é simplesmente a combinação de várias outras funções. Finalmente, iremos aplicar a supressão não-maximal da saída da rede para obter uma lista de *bounding boxes* que se aproxime da nossa lista inicial do treino.

Atenção, as implementações desse lab são simplificadas em comparação à literatura, pois o objetivo do lab é ensinar a essência, o básico, a intuição, os elementos fundamentais. Para uma aplicação no mundo real, utilize modelos e arquiteturas estado-da-arte consagrados <sub><sup>pelos sacerdotes do Deep Learning</sub></sup>.

Trivia: YOLO, *You Only Look Once*, nome da arquitetura de rede neural convolucional é inspirada pela gíria inglês *You Only Live Once*, utilizada geralmente quando alguém vai fazer alguma ação perigosa ou [alguma besteira](https://www.youtube.com/watch?v=dh6RB1RT9i0)

## Imports and data downloading

In [ ]:
!pip3 freeze | grep 'timm' || pip3 install -Uqq datasets==2.14.5 timm==0.9.7

Todos os nossos imports serão realizados automaticamente pela FastAI. Enquanto ma maior parte das bibliotecas pode ser uma má ideia utilizar um star import (`*`) pois irá poluir o seu ambiente, o FastAI toma cuidado de limitar o que é exportado (importado pelo `*`) por meio da definição da variável `__all__` dentro de um módulo. Assim, tudo que é exportado/importado por ele é intencional.

In [ ]:
from fastai.vision.all import *

from datasets import load_dataset
import matplotlib.patches as patches
from timm.models.layers import trunc_normal_, DropPath
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

Atenção, se por algum motivo você for mudar a pasta `/content` ou a flag de `RETRAIN`, mude na célula abaixo, pois ela é sobrescrita durante a correção (*read_only*) uma vez que para a correção funcionar o `base_path` tem que apontar para a pasta correta. Durante a correção, o `RETRAIN` será `False`.

In [ ]:
# dataset_lab read_only

! [ ! -d "/content/nanoyolo" ] && gdown -O /content/nanoyolo.zip "1H2w5WANOzmnn7OawBNAUtr0vhYb0e8P8" &&  unzip -q /content/nanoyolo.zip -d /content && rm /content/nanoyolo.zip
! [ ! -d "/content/nanoyolo" ] && wget -P /content/ "http://ia.gam.dev/cm203/23/lab06/nanoyolo.zip"  &&  unzip -q /content/nanoyolo.zip -d /content && rm /content/nanoyolo.zip
base_path = Path("/content/nanoyolo")
%cd /content

RETRAIN = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

E agora vamos baixar o dataset de detecção de terroristas e contra-terroristas do CS-GO! Estamos utilizando o [HuggingFace](https://huggingface.co/) 🤗 como repositório dos dados. Além dos dados, na realidade, ele é um dos maiores repositórios de redes neurais do mundo, disponibilizando os pesos das redes gratuitamente e de forma aberta. Ele também tem a sua própria biblioteca baseado no PyTorch e que lembra o FastAI, justamente por ter sido um dos seus fundadores, o Sylvain Gugger, o coautor que ajudou o Jeremy Howard a desenvolver a biblioteca do FastAI e escrever o livro e o curso.

Outra referência importante para a vida, é o [Papers With Code](https://paperswithcode.com/) onde os pesquisadores que escrevem artigos, principalmente pre-prints do arXiv, juntamente com a comunidade acadêmica e da indústria indexam os trabalhos mais relevantes em conjunto com a suas implementações no GitHub.

Devida à natureza altamente rápida do desenvolvimento de IA, a maior parte dos pesquisadores usam conferências em vez de journals para a publicação de seus trabalhos, mas antes disso ainda, o usual é publicar um pré-print no arXiv e espalhar o seu trabalho na comunidade, principalmente através do Twitter, onde naturalmente os artigos mais relevantes tendem a ser compartilhado mais frequentemente pela comunidade. Os principais pesquisadores, como [Yann LeCun](https://twitter.com/ylecun), o [Geoffrey Hinton](https://twitter.com/geoffreyhinton), o [Jeremy Howard](https://twitter.com/jeremyphoward), o [Sylvain Gugger](https://twitter.com/GuggerSylvain), o [Ilya Sutskever](https://twitter.com/ilyasut), o [Andrej Karpathy](https://twitter.com/karpathy), o . Eu também tenho minha conta no Twitter, [Gabriel A. Melo](https://twitter.com/gabruio), ainda não tenho nenhuma publicação relevante, mas pelo menos eu tento seguir os principais pesquisadores.

In [ ]:
cs_dataset = load_dataset("keremberke/csgo-object-detection", name="full").sort('image_id') # keep_in_memory=True

O bom dessa biblioteca é que em vez de armazenar as imagens como arquivos separados, ela utiliza uma estrutura de banco de dados (Apache Parquet), que diminui o overhead de ler vários arquivos pequenos. A forma como ele disponibiliza esses dados é de um dicionário especial:

In [ ]:
cs_dataset

Que nós podemos acessar tal qual um `dict` normal, observe que o `bbox` é uma lista de *tuplas* (na verdade são listas de tamanhos fixos) de 4 elementos onde cada tupla representa uma *bounding box* por meio da sua aresta $(x_0, y_0, w, h)$ do canto superior esquerdo $(x_0, y_0)$ e de sua altura e largura $(w, h)$:

In [ ]:
dado_treino_0 = cs_dataset['train'][1052]
dado_treino_0

E também vamos definir os nomes das classes, isto é, cada número inteiro do `category` na realidade está associado a um desses nomes:

In [ ]:
nomes_classes = ['Contra-Terrorista', 'Cabeça CT', 'Terrorista', 'Cabeça Terrorista']

Observe a primeira imagem com as suas respectivas bounding boxes desenhadas (reza a lenda que o código de plot tem parte da resposta da próxima questão):


*(não precisa entender o código abaixo)*

In [ ]:
def plot_dado(dado, desenha_bb=True, desenha_centros=False, grid=0):
    fig, ax = plt.subplots(dpi=100)
    ax.imshow(dado['image'])
    if grid:
        width, height = dado['image'].size
        ax.set_xticks(np.arange(0, width, grid))
        ax.set_yticks(np.arange(0, height, grid))
        ax.grid(True)
    else:
        ax.set_axis_off()
    if not desenha_bb:
        return
    cores = ['green', 'blue', 'red', 'pink']
    for bbox, cate in zip(dado['objects']['bbox'], dado['objects']['category']):
        x_vertice, y_vertice, largura, altura = bbox
        ax.add_patch(patches.Rectangle((x_vertice, y_vertice), largura, altura, linewidth=2, edgecolor=cores[cate], facecolor='none'))
        ax.text(x_vertice, y_vertice, nomes_classes[cate], c=cores[cate])
        if desenha_centros:
            plt.plot(x_vertice+largura/2, y_vertice+altura/2, color=cores[cate], marker='x')

plot_dado(dado_treino_0, grid=64)

## Montando o tensor de Saída Verdadeira (2 pontos)

**Explicação sobre o assunto**

Uma das grandes sacadas na detecção de objetos baseadas em um único estágio é justamente como definir um tensor de saída que tenha dimensões fixas (na realidade em função do tamanho da imagem), sendo que a nossa saída na realidade é uma lista de *bounding boxes* (BBs).

A primeira tentativa seria simplesmente limitar o tamanho máximo dessa lista, indicando com uma flag se o índice está vazio ou não. Mas se fôssemos fazer isso nós teríamos um outro problema (que é até mais difícil), que é a ordenação dos elementos dessa lista, que precisaria seguir uma regra bem definida e consistente, pois se não estaríamos penalizando a rede aleatoriamente pela ordem da saída que ela gera. Acontece que tanto definir fazer com que a rede aprende tal mecanismo de ordernação não é tão fácil quanto parece ...

Isso se torna evidente quanto temos muitas bounding boxes em uma mesma imagem, veja o exemplo abaixo (com 18 BBs):

In [ ]:
exemplo_muitos = cs_dataset['train'][1807]
exemplo_muitos['objects']['bbox'], exemplo_muitos['objects']['category']

Olhe a imagem abaixo e tente localizar os terroristas e os contra-terroristas:

In [ ]:
plot_dado(exemplo_muitos)

Outra solução ainda nessa linha seria simplesmente encontrar uma permutação dos elementos dessa lista que minimize a penalização (o dano, o gradiente) que iremos causar na rede, assim não precisaríamos nos preocupar com a ordem. A dificuldade agora está justamente em fazer esse match, que se torna cada vez mais difícil ($n^2$) com o aumento do tamanho máximo $n$ dessa lista.

Mas a solução natural é aproveitar a própria estrutura 2d espacial da imagem (entrada) e da saída (bounding boxes) para termos, enfiom, uma estrutura de tamanho bem definido sem se preocupar com ordenação, que na realidade já vem de graça pelas próprias dimensões espaciais! Isto é, associamos as bounding boxes a um grid espacial.

Observe o grid abaixo da YOLOv1, sendo que agora só temos uma única bounding box associada a cada posição do grid.

![YOLO tensor](https://www.researchgate.net/profile/Cedric_Perauer/publication/349929458/figure/fig7/AS:999598260768778@1615334205592/YOLOv4-output-tensor-29-The-width-and-height-depend-on-the-size-of-the-head_W640.jpg)

Então no final das contas, é como se estivéssemos transformando uma imagem de entrada RGB $(3, H, W)$ de altura $H$ e largura $W$ em uma outra *imagenzinha* de saída $(1 + 4 + C, h_g, w_g)$ que tem dimensões espaciais (largura $w_g$ e altura $h_g$) muito menores, mas com dimensões de informação adicionais sendo $1$ para indicar a presença de objeto, $4$ para indicar uma bounding box e $C$ para indicar a classe caso exista o objeto.

Por exemplo, veja abaixo uma imagem e um tensor de saída de um modelo super simples de 1 camada convolucional (entenda o shape abaixo):

In [ ]:
img = torch.ones(3, 128, 128)
modelo_simplorio = nn.Conv2d(in_channels=3, out_channels=1+4+10, kernel_size=16, stride=16)
saida = modelo_simplorio(img)
saida.shape

Vale lembrar que existem várias formas possíveis para essa realização, neste lab iremos implementar a mais simples, que é apenas uma bounding box por elemento do grid e o grid não tem sobreposição. Em outras palavras, uma bounding box está associada apenas em um elemento do grid, aquele que contiver o seu centro, e cada grid contém apenas uma bounding box.

Nas arquiteturas estado-da-arte é comum que um elemento do grid possa conter várias bounding boxes (justamente para poder tratar o caso de. Na própria YOLOv1 original um elemento do grid continha 2 BBs e em sua versão posterior conteve 5 anchor boxes (que são BBs de tamanhos já pré-definidos, *manjados*). É claro que no caso de haverem várias BBs voltamos no problema de fazer o match, só que agora é muito mais controlado (quantidade bem pequena, 2-5 BBs) do que que se fizéssemos na imagem inteira (dezenas, centenas de BBs).

Nesse caso as várias bounding boxes seriam mais dimensões sobre os canais de saída (entenda o shape abaixo):

In [ ]:
modelo_simplorio_2bbs = nn.Conv2d(in_channels=3, out_channels=(1+4)*2+10, kernel_size=16, stride=16)
saida2 = modelo_simplorio_2bbs(img)
saida2.shape

Além disso, outra ideia interessante é pemitir a sobreposição do grid ou ainda atribuição de uma mesma BB a vários elementos de grid próximos. Porque isso, você deve estar se perguntando?

Suponha, por exemplo, o caso em que o centro de uma BB caia exatamente (ou muito próximo) da borda entre um grid e outro. Dependendo do que a rede atribuir e de como você construiu o tensor, a rede poderá ser penalizada enormemente por um erro que deveria ser muito pequeno. Isso é ruim e era exatamente um dos problemas que queríamos evitar (só que ainda persiste em menor escala). Esse é um caso extremo, mas dá para intuir que seria interessante se os grids vizinhos pudessem compartilhar também parte dessa informação.

Para ajudar a rede a ser menos penalizada quando um objeto estiver bem na borda do grid, isto é, próximo de ser atribuído a um grid vizinho e fazer a rede errar, podemos criar um overlapping entre os próprios grids.

A forma exata desse overlapping depende da operação que foi empregada, sobretudo se houve ou não *padding same*. Em geral as operações que reduzem as dimensões espaciais não tem nenhum padding (*valid*) e usam stride igual ao tamanho do kernel, que em geral é 2x2. Então se para a última camada que reduz as dimensões espaciais ńos colocássemos um stride 1 no lugar de 2 teríamos grids que teriam overlapping de metade em cima do seus vizinho.

Além do overlapping podemos pensar da própria atribuição das BBs ao longo do grid, que em vez de atribuir apenas ao elemento do grid que contém o centro da BB, atribuir também aos vizinhos mais próximos ou outros que estejam contidos na BB. (Essa é uma ideia quase equivalente a anterior, mas só que agora em vez de atribuir a exatamente dois elementos do grid, atribuimos a mais)

Enfim, são outras possíveis realizações do grid. Mas também não se esqueçam que além do grid (que é o método de um único estágio), existem também métodos de dois estágios (localizar, *recortar* e classificar), que é ainda uma outra solução do problema original de detecção.

Essa é uma outra ideia interessante para o exame, adaptar uma arquitetura de YOLO para ter overlapping de grids e/ou atribuições múltiplas. Para quem quiser ser menos aventureiro, também aplicar uma arquitetura estado-da-arte de detecção de objetos no seu próprio dataset também é válido! Só tenha cuidado para não fazer as duas coisas ao mesmo tempo: ou você cria uma arquitetura nova em um dataset conhecido ou você usa uma arquitetura conhecida em um dataset *novo*, justamente para poder realizar comparações.

Veja abaixo como as dimensões espaciais aumentaram quase dobraram (é aquele $W_{final} = \lfloor{\frac{W_{img} - W_{kernel} + 1}{stride}}\rfloor +1$ que o Gabriel pembou com o +1 na aula).

In [ ]:
modelo_simplorio_overlapping = nn.Conv2d(in_channels=3, out_channels=(1+4)*2+10, kernel_size=16, stride=8)
saida3 = modelo_simplorio_overlapping(img)
saida3.shape

**Enunciado da Questão**

Implemente a função `monta_grid_detecta` abaixo, de acordo com a sua documentação. A ideia é implementar o grid descrito acima da forma mais simples possível: apenas uma BB por elemento do grid.

No nosso caso simplificado, o grid não tem overlapping, a bounding boxes deve ser
associado por onde o seu centróide cair no grid. A posição dela deve ser RELATIVA a esse
elemento do grid, sendo a origem no seu centro e sendo normalizado entre -1 e 1.
As larguras e as alturas da bounding boxes devem ser normalizadas pela norms_bb, que
representam a largura e altura média das BBs do conjunto de treino e com log.
As classes devem ser codificadas como one-hot encodding (class_one_hot_encoding).
Para um elemento do grid as classes devem ser ordenadas da seguinte forma:

$[P_{obj}, Xc_{norm}, Yc_{norm}, W_{norm}, H_{norm}, class_{one hot encoding}]$

$0 \leq P_{obj} \leq 1$ : probabilidade de haver objeto no grid

$-1 \leq X_{norm},Yc_{norm} < 1 $: posição do centroide da BB relativa ao centro do grid

$W_{norm},H_{norm}$ : logaritmos da altura e largura da BB normalizadas

Caso duas BBs caiam no mesmo grid, o comportamento esperado e que a última (pela 
ordem da lista) sobreescreva as mais antigas.

Os testes eles são feitos passo-a-passo, então uma dica que eu deixo para você é fazer tipo um TDD (Test-Driven Development) e ir implementando **CONSCIENTEMENTE** a função passo-a-passo (baby-steps) e verificando se o seu pequeno passo está certo, por meio do teste (não se esqueça de verificar se os testes antigos ainda funcionam)

**NÃO** use LLMs (ChatGPT) para pegar a resposta pronta. **NÃO** pesquise a resposta pronta na internet.

**Pode** olhar a documentação das bibliotecas (PyTorch, FastAI, mas todas as funções que você precisa está nas **dicas**) e **pode** (aconselhado) olhar o material de aula (slides e referências).

<details><summary><b>Dica para a resposta</b></summary>
<p>
    
Precisa de um for-loop para percorrer a lista de bounding boxes e de classes juntas, em python dá para você usar o operador `zip`, por exemplo: `for bbox, clas in zip(lista_bb, lista_classes):`

Então para cada elemento modifique o valor já nas posições dimensões (canais) especificados e já normalizados.

Funções da biblioteca pertinentes: `tensor`, `torch.zeros`, `torch.floor`, `torch.log`

Use slices se quiser economizar linhas: `bbox[2:]` entre outros ... e dá para fazer atribuição de múltiplas variáveis com o valor de um array/lista: `idx, idy = [0, 0]`. A minha solução ficou com 8 linhas
</p>
</details>

In [ ]:
# questao_montar_tensor autograded_answer

def monta_grid_detecta(lista_bb: Tensor, lista_classes: Tensor,
                       C: int, W: int, H: int, S: int, norms_bb: Tensor) -> Tensor:
    """ Cria o tesor da saída verdadeira definida pelas Bounding Boxes anotadas.
    
    Args:
        lista_bb: Lista de bounding boxes definidas pelos vértice x0, y0, wbb, hbb do
            topo esquerdo e por sua largura e altura
        lista_classes: Lista de classes associadas a cada bouding box respectivamente
        C: número de classes. O valores da lista_classes vão de 0 a C-1 inclusive.
        W: largura da imagem em pixels (width)
        H: altura da imagem em pixels (height)
        S: tamanho em pixels de cada quadrado do grid de saída (size)
        norms_bb: largura e altura médias das BBs, isto é, fatores de normalização
    
    Returns:
        Tensor de shape (1+4+C, H//S, W//S) que representa o grid de bounding boxes
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()

Se você usou uma LLM, escreva a sua conversa com ela aqui nesta própria célula de texto (copie a conversa inteira) ou exporte o link da conversa:

**Escreva aqui**



Primeiro vamos verifica se com a entrada vazia nós temos um tensor de tamanho correto e se as probabilidades de ter ou não um objeto também estão nulas (pois a lista de bounding boxes está vazia). Para isso você deve ter implementado a funcionalidade de inicialização do tensor de saída com os valores adequados (nulos para a probabilidade e qualquer valor para o resto).

Entenda o que cada linha do teste faz, por exemplo, `torch.linalg.norm(saida - esperado) < 𝜖` calcula a norma euclidiana da diferença entre a saída e o valor esperado, e verifica se isso é menor do que um número muito pequeno $\epsilon$

In [ ]:
# testa_1_questao_montar_tensor autograder_tests 0.2

grid01 = monta_grid_detecta(lista_bb=tensor([]), lista_classes=tensor([]), C=8, 
                            W=100, H=100, S=10, norms_bb=tensor([1., 1.]))

assert grid01.shape == (13, 10, 10)
assert grid01.dtype == torch.float32
assert torch.linalg.norm(grid01[0] - torch.zeros(10, 10)) < 1e-6


E agora verificando se você não inverteu largura por altura do grid (ainda para o caso da entrada vazia):

In [ ]:
# testa_2_questao_montar_tensor autograder_tests 0.2

grid02 = monta_grid_detecta(lista_bb=tensor([]), lista_classes=tensor([]), C=4, 
                            W=100, H=200, S=10, norms_bb=tensor([1., 1.]))

assert grid02.shape == (9, 20, 10)
assert torch.linalg.norm(grid02[0] - torch.zeros(20, 10)) < 1e-6


Vamos verificar a probabilidade de haver um objeto. Façamos um grid 1x1 com uma bounding box centrada exatamente no meio, a probabilidade deve ser igual a 100%: 

In [ ]:
# testa_3_questao_montar_tensor autograder_tests 0.1

grid03 = monta_grid_detecta(lista_bb=tensor([[0., 0, 77, 77]]), lista_classes=tensor([0]),
                            C=5, W=77, H=77, S=77, norms_bb=tensor([1., 1.]))

assert grid03.shape == (10, 1, 1)
assert grid03[0, 0, 0] == 1.

Aumetemos o tamanho do grid e vejamos se ainda está funcionando a probabilidade de haver objeto:

In [ ]:
# testa_3b_questao_montar_tensor autograder_tests 0.1

grid03b = monta_grid_detecta(lista_bb=tensor([[20., 20, 20, 20]]), lista_classes=tensor([3]),
                             C=4, W=100, H=100, S=20, norms_bb=tensor([1., 1.]))

probs = tensor([[0., 0., 0., 0., 0.],
                [0., 1., 0., 0., 0.],
                [0., 0., 0., 0., 0.],
                [0., 0., 0., 0., 0.],
                [0., 0., 0., 0., 0.]])

assert grid03b.shape == (9, 5, 5)
assert torch.linalg.norm(grid03b[0] - probs) < 1e-6

Voltando para o caso 1x1. Vamos verificar se a classe está correta:

In [ ]:
# testa_4_questao_montar_tensor autograder_tests 0.2

grid04 = monta_grid_detecta(lista_bb=tensor([[0., 0, 8, 8]]), lista_classes=tensor([0]),
                            C=5, W=8, H=8, S=8, norms_bb=tensor([1., 1.]))

classe_one_hot = tensor([1, 0, 0, 0, 0])

assert grid04.shape == (10, 1, 1)
assert torch.linalg.norm(grid04[5:, 0, 0] - classe_one_hot) < 1e-6

Vamos verificar o centro dessa BB, perceba que eu coloquei ela exatamente no quadrante do fundo direito do grid.

In [ ]:
# testa_5_questao_montar_tensor autograder_tests 0.2

grid05 = monta_grid_detecta(lista_bb=tensor([[40., 40, 40, 40]]), lista_classes=tensor([0]),
                            C=5, W=80, H=80, S=80, norms_bb=tensor([1., 1.]))

centro = tensor([0.5, 0.5])

assert grid05.shape == (10, 1, 1)
assert torch.linalg.norm(grid05[1:3, 0, 0] - centro) < 1e-6


E ver se você está fazendo as normalizações da bounding boxes corretamente, temos uma bounding box com 60 pixels de largura e 50 pixels de altura mas o tamanho médio das BBs do treino são de 30 pixels de largura e 20 pixels de altura. Portanto em termos relativos (multiplicativos) a nossa largura e altura normalizadas devem ser de 2 e de 2.5 respectivamente:

In [ ]:
# testa_6_questao_montar_tensor autograder_tests 0.2

grid06 = monta_grid_detecta(lista_bb=tensor([[0, 0, 60, 50]]), lista_classes=tensor([0]), C=5, 
                            W=80, H=80, S=80, norms_bb=tensor([30., 20.]))

largura_altura = torch.log(tensor([2, 2.5]))

assert grid06.shape == (10, 1, 1)
assert torch.linalg.norm(grid06[3:5, 0, 0] - largura_altura) < 1e-6

E finalmente realizamos o teste com dados reais. Você consegue olhar para as matrizes abaixo e enxergar os valores não nulos e entender porque deles serem assim? Olha na imagem abaixo.

In [ ]:
# testa_7_questao_montar_tensor autograder_tests 1

bbs_testa = tensor([
    [22.0, 0.0, 98.0, 88.0],
    [19.0, 0.0, 396.0, 415.0],
    [0.0, 166.0, 131.0, 249.0]])
categorias_testa = tensor([3, 2, 2])

probs = tensor(
    [[0., 1., 0., 0., 0., 0., 0.],
     [0., 0., 0., 0., 0., 0., 0.],
     [0., 0., 0., 0., 0., 0., 0.],
     [0., 0., 0., 1., 0., 0., 0.],
     [0., 1., 0., 0., 0., 0., 0.],
     [0., 0., 0., 0., 0., 0., 0.],
     [0., 0., 0., 0., 0., 0., 0.]])
x_norms = tensor(
    [[ 0.0000, -0.7812,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
     [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
     [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
     [ 0.0000,  0.0000,  0.0000, -0.2188,  0.0000,  0.0000,  0.0000],
     [ 0.0000, -0.9531,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
     [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
     [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])
h_norms = tensor(
    [[0.0000, 0.0394, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
     [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
     [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
     [0.0000, 0.0000, 0.0000, 1.5903, 0.0000, 0.0000, 0.0000],
     [0.0000, 1.0795, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
     [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
     [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
class_terror = tensor(
    [[0., 0., 0., 0., 0., 0., 0.],
     [0., 0., 0., 0., 0., 0., 0.],
     [0., 0., 0., 0., 0., 0., 0.],
     [0., 0., 0., 1., 0., 0., 0.],
     [0., 1., 0., 0., 0., 0., 0.],
     [0., 0., 0., 0., 0., 0., 0.],
     [0., 0., 0., 0., 0., 0., 0.]])

grid07 = monta_grid_detecta(
    lista_bb=bbs_testa, lista_classes=categorias_testa,
    C=4, W=448, H=448, S=64, norms_bb=tensor([55.5, 84.6]))

assert grid07.shape == (9, 7, 7)
assert torch.linalg.norm(grid07[0] - probs) < 1e-3
assert torch.linalg.norm(grid07[1] - x_norms) < 1e-3
assert torch.linalg.norm(grid07[4] - h_norms) < 1e-3
assert torch.linalg.norm(grid07[7] - class_terror) < 1e-6


Tem outras dimensões y e w que não coloquei os tensores, mas dê uma olhada na imagem e veja se faz sentido:

In [ ]:
dado_testa = cs_dataset['train'][1063]
if RETRAIN:
    plot_dado(dado_testa, grid=64, desenha_centros=True)

## Definindo a Arquitetura (2 pontos)

**Explicação sobre o assunto**

No lab passado nós vimos uma arquitetura de rede neural para classificação de imagens e também como realizar um transfer learning. Se você tiver realmente entendido o lab passado, você já deve conseguir enxergar uma pequena modificação no final daquela arquitetura que faz com que o tensor de saída dela seja um *grid*. Mesmo que não tenha percebido isso no lab passado, não tem problema, pois também explicaremos novamente neste lab.

O ponto fundamental para essa compreensão é entender a operação de convolução e observar que a rede neural que contruímos é composta puramente de operações convolucionais (e seus equivalentes matemáticos que operam elemento-a-elemento, ativações, normalizações). Dessa forma, a cada camada nós temos uma entrada que é uma *imagem* (ou batch de imagens) da forma $(\ldots, C_{in}, H_{in}, W_{in})$, que tem $C_{in}$ canais (não necessariamente de cores, mas de ativações que não sabemos exatamente o que significam), e dimensões espaciais de altura $H_{in}$ e de largura $W_{in}$. A nossa operação de convolução vai agir de forma local sobre as dimensões espaciais e de forma *global* sobre os canais (a priori todos os canais de entrada estão conectados com os de saída).

Dessa forma, após uma convolução ou uma sequência de convoluções, ainda continuamos com um tensor da forma $(\ldots, C_{out}, H_{out}, W_{out})$, com dimensões de canais de dimensões espaciais.

Também é importante que você entenda a forma como a informação se propaga pelas dimensões espaciais e como as larguras e as alturas mudam (se permanecem iguais graças a um *padding same*, ou se diminuem nas bordas, ou ainda por um fator inteiro graças ao *stride*).

Em especial, como as dimensões espacial são reduzidas, se há overlapping ou se há *distorções* nas bordas. Por exemplo, a questão anterior do grid precisa de convoluções com *padding same* (que não mude os tamanhos espaciais) justamente para poder fazer uma projeção direta (usar a regra de 3, só dividir pelo S) do grid sobre a imagem de entrada. E ainda para não ter overlapping, é necessário que as operações que reduzam a dimensão tenham stride igual ao tamanho do kernel, em geral 2x2 com stride 2 que reduz exatamente pela metade, e sem nenhum padding (apenas *valid*).  

O código abaixo ([Github](https://github.com/facebookresearch/ConvNeXt)) é do [Meta AI Research](https://ai.meta.com/research/), do modelo ConvNext, que é um tipo de ResNet mais moderna, com normalização por camadas (*LayerNorm*), convoluções puramente espaciais (com tantos grupos quanto canais, *depthwise*) e dropout nas próprias conexões residuais (*DropPath*).

Atenção, a `ConvNeXt` abaixo é apenas para classificação (saída logits da softmax), o trabalho de vocês vai ser mudar as últimas camadas para permitir a detecção (saída grid).

Só para aquecer vamos começar pelo o `LayerNorm` abaixo. Eles fizeram duas implementações diferentes na mesma classe, uma para o caso unidimensional, que são os `channels_last` e outra para o caso 2d que trabalhamos no PyTorch, com os `channels_first`. Ambos são uma normalização subtrai a média e divide pelo desvio padrão, e que tem parâmetros de escala (`weight` e `bias`) a serem treinados. A única diferença está nas dimensões em que as operações são realizadas. No caso 2d (`channels_first`), ele reduz a dimensão 1 que são os canais, o `C` do (B,C,H,W) para calcular a média e o desvio padrão, assim não há influência espacial nessa normalização (cada pixel é separado e depende apenas dos seus canais).

*(não precisa entender o código abaixo (que os pesquisadores implementaram), mas é importante que você entenda a explicação acima*

In [ ]:
class LayerNorm(nn.Module):
    r""" LayerNorm that supports two data formats: channels_last (default) or channels_first. 
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with 
    shape (batch_size, height, width, channels) while channels_first corresponds to inputs 
    with shape (batch_size, channels, height, width).
    """
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError 
        self.normalized_shape = (normalized_shape, )
    
    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x

Em seguida temos o bloco da rede convolucional do MetaAI, que é bem semelhante ao que vocês implementaram no lab passado. A principal diferença está em uma regularização da conexão residual, por meio do `DropPath`, que tal qual o *DropOut*, tem uma probabilidade de zerar completamente os valores, só que agora não das ativações dos neurônios, mas da própria conexão residual.

Perceba que as dimensões de entrada são exatamente as mesmas de saída, ele não altera nem a quantidade de canais (apesar de que internamente ele aumenta 4 vezes e depois diminui os canais para ficar igual da entrada), nem as dimensões espaciais (pois usa padding).

Algo que eu achei bem interessante foi o fato deles usarem uma camada densa no lugar de uma conv1x1. É matematicamente equivalente, só que eles tiveram mais trabalho para fazer reshape antes e reshape depois. Talvez eles tenham feito isso por performance (por que se eles tivessem deixado a conv1x1 nem precisariam ter implementado a LayerNorm com channels_last). Algo muito ruim (do ponto de vista de engenharia) que eles fizeram foi nomear uma variável `input`, pois é uma palavra reservada do Python.

*(não precisa entender o código abaixo (que os pesquisadores implementaram), mas é importante que você entenda a explicação acima*

In [ ]:
class Block(nn.Module):
    r""" ConvNeXt Block. There are two equivalent implementations:
    (1) DwConv -> LayerNorm (channels_first) -> 1x1 Conv -> GELU -> 1x1 Conv; all in (N, C, H, W)
    (2) DwConv -> Permute to (N, H, W, C); LayerNorm (channels_last) -> Linear -> GELU -> Linear; Permute back
    We use (2) as we find it slightly faster in PyTorch
    
    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim) # depthwise conv
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim) # pointwise/1x1 convs, implemented with linear layers
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)), 
                                    requires_grad=True) if layer_scale_init_value > 0 else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1) # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.permute(0, 3, 1, 2) # (N, H, W, C) -> (N, C, H, W)

        x = input + self.drop_path(x)
        return x

Finalmente temos a definição final da ConvNeXt. O código está exatamente como reportado pelos pesquisadores, eu não alterei nada (é para vocês verem como os pesquisadores de IA escrevem código para divulgação, que em geral leva mais trabalho para entender).

Em termos simplificado ele instância 4 camadas de *downsampling* e 4 conjunto de camadas de blocos (cada conjunto tem múltiplos blocos), e por último tem o `self.head` e o `self.norm` que nós iremos alterar.

É importante que você conheça que o *downsamplings* diminuem as dimensões espaciais, respectivamente por 4, por 2, por 2 e por 2. Assim, a redução total é de 32 vezes, que gera o nosso grid (o tamanho dele). Entendeu o porquê do tamanho do grid?

*(não precisa entender o código abaixo [(que os pesquisadores implementaram)](https://github.com/facebookresearch/ConvNeXt/blob/main/models/convnext.py), mas é importante que você entenda a explicação acima*

In [ ]:
class ConvNeXt(nn.Module):
    r""" ConvNeXt
        A PyTorch impl of : `A ConvNet for the 2020s`  -
          https://arxiv.org/pdf/2201.03545.pdf

    Args:
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        depths (tuple(int)): Number of blocks at each stage. Default: [3, 3, 9, 3]
        dims (int): Feature dimension at each stage. Default: [96, 192, 384, 768]
        drop_path_rate (float): Stochastic depth rate. Default: 0.
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
        head_init_scale (float): Init scaling value for classifier weights and biases. Default: 1.
    """
    def __init__(self, in_chans=3, num_classes=21841, 
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], drop_path_rate=0., 
                 layer_scale_init_value=1e-6, head_init_scale=1.,
                 ):
        super().__init__()

        self.downsample_layers = nn.ModuleList() # stem and 3 intermediate downsampling conv layers
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        for i in range(3):
            downsample_layer = nn.Sequential(
                    LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        self.stages = nn.ModuleList() # 4 feature resolution stages, each consisting of multiple residual blocks
        dp_rates=[x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))] 
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j], 
                layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]

        self.norm = nn.LayerNorm(dims[-1], eps=1e-6) # final norm layer
        self.head = nn.Linear(dims[-1], num_classes)

        self.apply(self._init_weights)
        self.head.weight.data.mul_(head_init_scale)
        self.head.bias.data.mul_(head_init_scale)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)

Agora esse método é importante (o nome dele é arbitrário), mas prestem atenção pois vocês devem chamá-lo para facilitar a sua vida. Entenda o código abaixo.

Esse método realiza a propagação direta até o ponto em que o tensor `x` ainda mantém as suas dimensões espaciais. Ou seja, ele passa por todas as camadas anteriores, que reduz por um fator de 32 as dimensões espaciais e aumenta os canais para 768 (no modelo *tiny*).

In [ ]:
@patch_to(ConvNeXt)
def forward_features(self: ConvNeXt, x: Tensor) -> Tensor:
    for i in range(4):
        x = self.downsample_layers[i](x)
        x = self.stages[i](x)
    return x

Esse método é o `forward` que vocês já devem estar habituados (o nome dele é importante para o PyTorch poder chamá-lo no __call__ do objeto).

Observe que a entrada é uma imagem, um tensor de shape $(B, 3, H, W)$. Em seguida ele chama o método da célula acima para obter um tensor de de shape $(B, 768, H//32, W//32)$ que foi o resultado de muitas convoluções.

Depois ele realiza uma redução das dimensões espaciais, fazendo uma média desses valores. E por fim utiliza uma camada densa (completamente conectada, a `head`) para calcular os *logits* de saída.

In [ ]:
@patch_to(ConvNeXt)
def forward(self: ConvNeXt, x: Tensor) -> Tensor:
    x = self.forward_features(x)
    x = self.norm(x.mean([-2, -1])) # global average pooling, (N, C, H, W) -> (N, C)
    x = self.head(x)
    return x

Assim, temos a ConvNeXt completamente definida (todos as suas camadas, blocos e operações).

Vamos baixar os parâmetros já treinados no ImageNet mais recente de 22 mil imagens (22k no lugar do 1k). É do repositório do Facebook (Meta AI):

In [ ]:
url = "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_22k_224.pth"
classification_weights = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)["model"]

Vamos instanciar a rede deles e carregar os seus pesos já treinados para classificação.

In [ ]:
model = ConvNeXt()
model.load_state_dict(classification_weights)

Perceba como é a saída antes de passar pelas últimas camadas `head`, para uma imagem de shape `(1, 3, 416, 416)`

In [ ]:
img_rand = torch.rand((1, 3, 416, 416))
features = model.forward_features(img_rand)
features.shape

E como ela fica no final:

In [ ]:
output_final_classificacao = model(img_rand)
output_final_classificacao.shape

Enfim, vamos definir a nossa própria rede convolucional de detecção, a `ConvDect`! Baseada na estrutura da `ConvNeXt`

Perceba que eu deletei as camadas `head` e `norm` da rede de classificação. Criei um `bloco_final1` que não altera o shape do tensor de entrada, mas que é apenas a aplicação de uma conv3x3 com padding same que age independemente para cada canal, isto é, puramente espacial (pois `groups==channels`) seguida de uma conv1x1 que age apenas nos canais (768). Todas essas convoluções tem função de ativação GELU após as ativações. Essa sequência de convoluções é repetiva por 4 vezes.

Deve haver uma conexão residual após o `bloco_final1`, o que é simples uma vez que o shape não mudou. Em seguida aplica-se a camada `apos_res` após a conexão residual para poder reduzir a quantidade de canais (reduz de 768 para 192). Essa camada também deve ter função de ativação GELU.

Novamente há o `bloco_final2` que é análogo ao 1, só que agora são menos canais (192). Há também a última convolução que altera de a quantidade de canais para a saída esperada: 1 probabilidade, 2 posições x/y normalizados, 2 larguras/alturas normalizadas e `num_classes` logits das probabilidades de pertencer à classe. A probabilidade deve ter função de ativação softmax e os x/y normalizados devem ter função de ativação tangente hiperbólica.

DISCLAIMER: Essas camadas que eu criei são arbitrárias mas motivadas na arquitetura da ConvNeXt (poderia ser diferente sem problemas), mas você deve implementá-las exatamente como na descrição para passar nos testes. Por exemplo, usar uma conv3x3 puramente espacial e depois uma conv1x1 tem menos parâmetros e computação do que usar uma conv3x3 normal.

In [ ]:
class ConvDect(ConvNeXt):
    """ Rede Convolucional para Detecção de Objetos, baseada na ConvNext que
    era uma rede apenas de classificação.
    
    Args:
        num_classes (int): Number of classes for classification head.
    """
    def __init__(self, num_classes):
        super().__init__(num_classes=num_classes)
        del self.norm
        del self.head
        self.bloco_final1 = nn.Sequential(
            *[nn.Sequential(
                nn.Conv2d(768, 768, 3, padding='same', groups=768),
                nn.GELU(),
                nn.Conv2d(768, 768, 1),
                nn.GELU())
            for i in range (4)]
        )
        self.apos_res = nn.Conv2d(768, 192, 1)
        self.bloco_final2 = nn.Sequential(
            *([nn.Sequential(
                nn.Conv2d(192, 192, 3, padding='same', groups=192),
                nn.GELU(),
                nn.Conv2d(192, 192, 1),
                nn.GELU())
            for i in range (4)] +
           [nn.Conv2d(192, 1 + 4 + num_classes, 1)])
        )

**Enunciado da Questão**

Implemente a função `forward` abaixo na classe `ConvDect` que é a nossa própria rede de detecção baseada na do MetaAI.

Há uma conexão residual depois do `bloco_final1` e antes do `apos_res`
A saída do apos_res deve passa por uma função de ativação GELU
A saída final deve ter função de ativação Sigmóide para o canal 0 ($P_{obj}$)
e ativação Tangente Hiperbólica para os canais 1 e 2 ($Xc_{norm}, Yc_{norm}$).

**NÃO** use LLMs (ChatGPT) para pegar a resposta pronta.

**Pode** olhar a documentação das bibliotecas (PyTorch, FastAI, mas todas as funções que você precisa está nas **dicas**) e **pode** (aconselhado) olhar o material de aula (slides e referências).

<details><summary><b>Dica para a resposta</b></summary>
<p>
Basta você chamar os blocos e a camada que eu declarei. Não se esqueça da conexão residual nem das funções de ativação `F.gelu`, `F.sigmoid`, `F.tanh`.
A minha solução ficou com 5 linhas.

Outras funções da biblioteca interessantes: `torch.cat`
</p>
</details>

In [ ]:
# questao_arquitetura autograded_answer

@patch_to(ConvDect)
def forward(self: ConvDect, x: Tensor) -> Tensor:
    """ Realiza a propagação direta da rede, utiliza todas as camadas da
    rede antiga (até a parte com dimensões espaciais) e também todas as
    camadas definidas posteriores no método init.

    Args:
        x: tensor de entrada (imagem de entrada) de shape (B, 3, H, W)

    Returns:
        tensor de grid com shape (B, 5+num_classes, H//32, W//32)
    """
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()

Se você usou uma LLM, escreva a sua conversa com ela aqui nesta própria célula de texto (copie a conversa inteira) ou exporte o link da conversa:

**Escreva aqui**

Verifica se o shape de saída está correto.

In [ ]:
# testa_1_questao_arquitetura autograder_tests 0.2

img_test0 = torch.rand(1, 3, 416, 448)
model = ConvDect(40)

saida0 = model(img_test0)

assert saida0.shape == (1, 45, 13, 14)


Verifica se as funções de ativação realmente estão limitando as saídas conforme o esperado

In [ ]:
# testa_1b_questao_arquitetura autograder_tests 0.3

img_test0b = torch.rand(4, 3, 448, 448) * 4 - 2
model = ConvDect(4)

saida0b = model(img_test0)

assert torch.all(saida0b[:, 0] <= 1)
assert torch.all(saida0b[:, 0] >= 0)

assert torch.all(saida0b[:, 1:3] <= 1)
assert torch.all(saida0b[:, 1:3] >= -1)


Verifica se todos os sub-módulos pertinentes foram chamados, isto é, se os gradientes não estão nulos

In [ ]:
# testa_2_questao_arquitetura autograder_tests 0.5

img_test2 = torch.rand(1, 3, 448, 448) * 4 - 2
model = ConvDect(4)

saida2 = model(img_test2)
saida2.sum().backward()

for param in model.parameters():
    assert torch.linalg.norm(param.grad) > 0


Testa na rede final já treinada em um dado do nosso dataset (atenção para a conexão residual)

In [ ]:
# testa_3_questao_arquitetura autograder_tests 1

img_test3 = IntToFloatTensor()(ToTensor()(
    PILImage(cs_dataset['train'][1063]['image'])))[None]
model = ConvDect(4)
model.load_state_dict(torch.load(base_path/'treinado.pth', map_location="cpu"))

saida3 = model(img_test3)

assert saida3.shape == (1, 9, 13, 13)
assert torch.linalg.norm(saida3[0, 0] - tensor(
   [[0.  , 0.  , 0.1 , 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
    [0.  , 0.02, 0.95, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
    [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
    [0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
    [0.  , 0.  , 0.  , 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
    [0.  , 0.  , 0.  , 0.01, 0.01, 0.  , 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
    [0.  , 0.  , 0.  , 0.  , 0.1 , 0.01, 0.16, 0.09, 0.  , 0.  , 0.  , 0.  , 0.  ],
    [0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
    [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
    [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
    [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
    [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
    [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])).item() < 0.02


Para o nosso treinamento iremos fazer um transfer learning a partir da rede de classificação já treinada congelar todos os outros parâmetros a menos dos nossos que fizemos:

In [ ]:
def transfer_learning(rede: ConvDect, pesos: dict[str, Tensor]):
    rede.load_state_dict(pesos, strict=False)
    for module in rede.modules():
        module.eval()
        for param in module.parameters():
            param.requires_grad = False
    for module in [rede.bloco_final1, rede.apos_res, rede.bloco_final2]:
        module.train()
        for param in module.parameters():
            param.requires_grad = True

No mundo real é melhor que você faça o transfer learning de uma rede treinada já em detecção, e não em classificação como estamos fazendo aqui. O intuito desse lab é mostrar para vocês justamente como funciona uma rede de detecção single-stage (tipo YOLO) e a forma mais fácil é partir justamente de uma rede que já faz classificação.

## Definindo a Função Custo (2 pontos)

**Explicação sobre o assunto**

Agora que já temos a nossa saída esperada e a nossa rede que calcula a saída estimada, precisamos de uma função custo para poder compará-las e obter um valor ÚNICO para poder fazer a retropropagação!

Pelo fato de termos apenas uma BB por elemento do grid, fica bem mais fácil, não precisamos ficar nos preocupando em dar match de BB. Lembrando que no match nós alteramos o valor da saída esperada de forma a penalizar menos a rede, assim o processo do match em si não precisa ser retropropagado (alteramos apenas um tensor que não requer gradiente). É como se o *professor* fosse mais compreensivo com o aluno durante a correção, isto é, o aluno deu uma resposta que não é exatamente igual ao que o professor esperava mas que não está essencialmente errada, só precisa de um outro *ponto de vista*.

Mas no nosso caso que temos apenas uma BB fica mais simples, basta comparar os valores diretamente. Mas é importante entender que cada valor (probabilidade, posição, largura, probabilidade de classes) são comparados de uma forma diferente e com pesos (weights) diferentes, caso queiramos que a rede seja mais sensível para localização ou para a classificação.

Por exemplo as posições X, Y, podemos simplesmente aplicas uma MSE, que é simplesmente subtrair o valor verdadeiro y do valor estimado ŷ e elevar ao quadrado. Uma coisa interessante a se notar é a forma de combinar esses custos ao longo das dimensões (sobretudo dos batches), que pode ser um somatório ou uma média. Além disso, para as funções custo do PyTorch dá para passar `reduction='none'` que não faz a redução das dimensões:

In [ ]:
y, ŷ = torch.rand(3, 3), torch.rand(3, 3)
F.mse_loss(ŷ, y, reduction='none'), (ŷ - y)**2

Por *default* a redução é a média `reduction='mean'` e significa a aplicação de um `torch.mean` sobre o resultado anterior.

In [ ]:
F.binary_cross_entropy(torch.rand(3, 4), torch.rand(3, 4))

A `F.cross_entropy` calcula a entropia cruzada sobre os *logits*, como vimos no lab passado (cuidado com a ordem de ŷ, e y que importa, pois é probabilidade vezes o log). Agora também vamos aplicar a

Veja como a segunda dimensão (dim=1) é reduzida, no exemplo abaixo era $(3, 4, 6, 8)$ que virou $(3, 6, 8)$:

In [ ]:
F.cross_entropy(torch.rand(3, 4, 6, 8), torch.rand(3, 4, 6, 8), reduction='none').shape

A nossa loss completa é uma função perda multi-tarefa (*multi-task*) que utiliza entropia binária (BCE - *binary cross entropy*) para o problema da classificação binária sobre as probabilidades de haver objeto ou não $p_{obj}$; erro quadrático médio (*MSE*) para os problemas de localização e de estimativa do tamanho da BB; e entropia cruzada (*CE - cross entropy*) para o problema de classificação entre as classes. Cada problema é ponderado por um fator $\lambda$ que indica a importância relativa dele. Além disso, as outras losses só fazem sentido se realmente houver algo no grid (são condicionados ao valor de $p_{obj}$ verdadeiro). Isto é, os outros valores (posição, tamanho, classe) não importam se não houver nada no grid de verdade (imagem fazia, por exemplo).

A formulação matemática da loss total está abaixo, mas não recomendo que você implemente conforme essa fórmula, mas que use já cada loss pronta do PyTorch `F.`

$L = \frac{1}{B H W} \sum_B \sum_H \sum_W \left ( \\
-\lambda_{BCE} p_{obj} \log{\hat{p}_{obj}} - \lambda_{BCE} (1-p_{obj}) \log(1-\hat{p}_{obj}) \\
+p_{obj}\frac{\lambda_{pos}}{2} (\hat{x}-x)^2 + p_{obj}\frac{\lambda_{pos}}{2} (\hat{y}-y)^2 \\
+p_{obj}\frac{\lambda_{size}}{2} (\hat{w}-w)^2 + p_{obj}\frac{\lambda_{size}}{2} (\hat{h}-h)^2 \\
-p_{obj}\frac{\lambda_{class}}{C} \sum_C p_{c} \log{\hat{p}_{c}} \\
\right )$

Por exemplo: (na sua implementação você tem que ser mais esperto do que esse exemplo)

In [ ]:
2 * F.mse_loss(ŷ, y) + 4 * F.cross_entropy(ŷ, y)

**Enunciado da Questão**

Implemente a função `loss_func` abaixo, que é interna à `nanoyolo_loss`, que calcula a *distância* entre o grid verdadeiro e o grid esperado.

**FAÇA VETORIZADO**

Essa questão com os testes também ficou boa para fazer com TDD (*Test-Driven Development*).

**NÃO** use LLMs (ChatGPT) para pegar a resposta pronta. **NÃO** pesquise a resposta pronta na internet.

**Pode** olhar a documentação das bibliotecas (PyTorch, FastAI, mas todas as funções que você precisa está nas **dicas**) e **pode** (aconselhado) olhar o material de aula (slides e referências).

<details><summary><b>Dica para a resposta</b></summary>
<p>

Use os bizus das explicações acima, você pode implementar a sua própria loss, mas recomendo usar o `F.binary_cross_entropy`, o `F.mse_loss` e o `F.cross_entropy`, `torch.mean`. Seja esperto no broadcasting e nos slices.

A minha implementação vetorizada ficou com 4 linhas, uma para cada tipo de loss, é apenas um return. Seja esperto no broadcasting e nos slices.
</p>
</details>

In [ ]:
# questao_loss_func autograded_answer


def nanoyolo_loss(bce_w=1., pos_w=1., size_w=1., class_w=1.):
    """Retorna a loss function combinada para a classificação binária (BCE)
    na probabilidade de haver objeto p_obj, para a localização e tamanho (MSE),
    e para a classificação (CE), ponderados pelos seus respectivos pesos. As
    de MSE e CE são ponderadas pelo p_obj verdadeiro. A redução é a média das
    dimensões.
    
    Args:
        bce_w: Fator de peso sobre a entropia binária (de probabilidade)
        pos_w: Fator de peso sobre o MSE da posição
        size_w: Fator de peso sobre o MSE da largura/altura da BB
        class_w: Fator de peso para a entropia cruzada de classificação
    
    Returns:
        Tensor de rank 0 que é loss total
    """
    def loss_func(ŷ: Tensor, y: Tensor) -> Tensor:
        # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
        raise NotImplementedError()
    return loss_func

Se você usou uma LLM, escreva a sua conversa com ela aqui nesta própria célula de texto (copie a conversa inteira) ou exporte o link da conversa:

**Escreva aqui**

Para uma saída estimada $\hat{y}$ que é exatamente igual à saída verdadeira o resultado dessa função deve ser zero. Além disso, deve ser um tensor de rank 0 (escalar do PyTorch).

No caso abaixo testamos com tudo zerado, pois as probabilidades, além de serem iguais, devem ter valor 0 ou 1 para que as entropias sejam nulas.

In [ ]:
# testa_1_questao_loss_func autograder_tests 0.1

y = torch.zeros(3, 15, 8, 8)
ŷ = torch.zeros(3, 15, 8, 8, requires_grad=True)

criterion = nanoyolo_loss(1, 1, 1, 1)

loss = criterion(ŷ, y)

assert loss.shape == torch.Size([])
assert abs(loss) < 1e-6


A loss deve ter um gradiente e ser retropropagável até $\hat{y}$

In [ ]:
# testa_1b_questao_loss_func autograder_tests 0.1

y1 = torch.rand(3, 15, 8, 8)
ŷ1 = torch.rand(3, 15, 8, 8, requires_grad=True)
criterion1b = nanoyolo_loss(1, 1, 1, 1)

calculated_loss1b = criterion1b(ŷ1, y1)
calculated_loss1b.backward()

assert calculated_loss1b.requires_grad
assert torch.linalg.norm(ŷ1.grad) > 0


Vamos verificar se apenas quando as probabilidade são diferente a loss é realmente a da Entropia Cruzada Binária (`binary_cross_entropy`)

In [ ]:
# testa_2_questao_loss_func autograder_tests 0.2

y2 = torch.zeros(3, 15, 8, 8)
ŷ2 = torch.zeros(3, 15, 8, 8)

y2[:, 0] = torch.rand(8, 8)
ŷ2[:, 0] = torch.rand(8, 8)
bce_loss = F.binary_cross_entropy(ŷ2[:, 0], y2[:, 0])

criterion2 = nanoyolo_loss(3, 1, 1, 1)
loss2 = criterion2(ŷ2, y2)

assert abs(loss2 - 3 * bce_loss) < 1e-6


Agora quando a localização do centroide estiver errada:

In [ ]:
# testa_3_questao_loss_func autograder_tests 0.2

y3 = torch.zeros(4, 10, 9, 9)
y3[:, 0] = 1
ŷ3 = y3.clone().detach()

y3[:, 1:3] = torch.rand(2, 9, 9)
ŷ3[:, 1:3] = torch.rand(2, 9, 9)
mse_loss = F.mse_loss(ŷ3[:, 1:3], y3[:, 1:3])

criterion3 = nanoyolo_loss(1, 7, 1, 1)
loss3 = criterion3(ŷ3, y3)

assert abs(loss3 - 7 * mse_loss) < 1e-6


Mas o cálculo de todos os outros valores está condicionado às probabilidades verdadeiras

OBS: Não se esqueça de testar a loss de MSE da largura e da altura também

In [ ]:
# testa_4_questao_loss_func autograder_tests 0.2

y4 = torch.zeros(4, 9, 6, 6)
ŷ4 = torch.zeros(4, 9, 6, 6)

ŷ4[:, 1:] = torch.rand(4, 8, 6, 6)

criterion4 = nanoyolo_loss(1, 1, 1, 1)
loss4 = criterion4(ŷ4, y4)

assert abs(loss4) < 1e-6


Agora para a classificação:

In [ ]:
# testa_5_questao_loss_func autograder_tests 0.2

y5 = torch.zeros(5, 9, 6, 6)
y5[:, 0] = 1
ŷ5 = y5.clone().detach()

y5[:, 5:] = torch.rand(5, 4, 6, 6)
ŷ5[:, 5:] = torch.rand(5, 4, 6, 6)
ce_loss = F.cross_entropy(ŷ5[:, 5:], y5[:, 5:])

criterion5 = nanoyolo_loss(1, 1, 1, 666)
loss5 = criterion5(ŷ5, y5)

assert abs(loss5 - 666 * ce_loss) < 0.01


Juntando tudo ao mesmo tempo:

In [ ]:
# testa_6_questao_loss_func autograder_tests 1

y6 = tensor([[0.4108, 0.1717, 0.0069, 0.1393, 0.1146, 0.4914, 0.8634],
[0.8840, 0.3539, 0.6189, 0.7642, 0.7592, 0.1080, 0.7073]]).reshape(2, 7, 1, 1)
ŷ6 = tensor([[0.8075, 0.1843, 0.3749, 0.3511, 0.9455, 0.5990, 0.9024],
[0.2924, 0.8519, 0.9057, 0.8252, 0.5625, 0.7529, 0.9267]]).reshape(2, 7, 1, 1)

criterion6 = nanoyolo_loss(1, 2, 3, 4)
loss6 = criterion6(ŷ6, y6)

assert abs(loss6 - 3.1717855) < 1e-5


## Supressão Não-Maximal (4 pontos)

**Explicação sobre o assunto**

Agora chegamos na última questão! Já que sabemos como construir o tensor a partir de uma lista de BBs, temos que saber fazer o inverso também: construir uma lista de BBs a partir de um tensor de saída.

Só que nós estaremos trabalhando com a saída estimada da rede, e não com a saída esperada que nós construimos lá em cima perfeitamente (apesar de que podemos usá-la para fazer um teste). Dessa forma, teremos muito ruído! Sobretudo probabilidades que não serão nem 0 nem 1, mas algo intermediário.

Assim como primeiro passo, nós temos que descartar todos os elementos do grid que tiverem probabilidade menor do que 1 (operação abaixo apenas ilustrativa).

In [ ]:
torch.rand((3, 4)) < 0.3

Depois devemos des-normalizar os valores de posição e de tamanho da bounding box (operação abaixo apenas ilustrativa, faça o inverso da questão 1), para obter o seu tamanho em pixels:

In [ ]:
torch.exp(torch.rand(2, 3, 4)) * tensor([[[2]],[[3]]])

Só depois disso que então aplicamos a supressão não-maximal, que consiste, por classe, encontrar a BB de maior probabilidade e ir iterando sobre as outras. Deve-se descartar as bounding boxes que tenham grande sobreposição com as outras.

Essa métrica de sobreposição nós chamamos de Intersection over Union (IoU) e pode ser facilmente calculada abaixo encontrando os vértices do topo esquerdo (max sobre os vértices mínimos) e do fundo direito (min sobre os vertices máximos), conforme descrito na figura abaixo

![IoU](https://ia.gam.dev/cm203/23/lab06/iou.png)

<details><summary><b>CLIQUE EM MIM (exemplos DE IoU para matching) </b></summary>
<p>
<img src="https://b2633864.smushcdn.com/2633864/wp-content/uploads/2016/09/iou_examples.png?lossy=2&strip=1&webp=1" alt="IoU"/>
</p>
</details>

O função IoU abaixo implementa essa operação passo-a-passo (pode servir de inspiração para os broadcastings da sua des-normalização), ela recebe duas bounding-boxes na forma de lista de floats e retorna um escalar.

In [ ]:
def IoU(bbox1, bbox2):
    boxes = np.vstack([bbox1, bbox2])        #  (2, 4)
    areas = np.prod(boxes[:, 2:4], axis=-1)  #    (2,)
    topo_esq = np.max(boxes[:, 0:2], axis=0) #    (2,)
    fundo_dir = np.min(boxes[:, 0:2]+boxes[:, 2:4], axis=0)
    if np.any(fundo_dir <= topo_esq):
        return 0
    intersection = np.prod(fundo_dir-topo_esq)
    union = np.sum(areas) - intersection
    return intersection / union

IoU([0., 0, 20, 30], [5., 0, 20, 30])

Dica para facilitar a sua implementação:

Na realidade, o processo de descarte do primeiro passo já pode ser realizado de forma implícita ao percorrer o loop dos valores de probabilidade já ordenados. Veja um exemplo do loop abaixo com o sort que lista os valores, o seu índice na ordem crescente:

In [ ]:
for valores, indices in zip(*torch.sort(torch.rand(2,2).flatten())):
    print(valores, indices)

Os valores das posições `xy` do grid já são disponibilizados no início da função abaixo, e recomendo utilizá-lo para o cálculo da desnormalização.

In [ ]:
torch.stack(torch.meshgrid(torch.arange(3), torch.arange(4), indexing='ij')[::-1])

**Enunciado da Questão**

Implemente a função `recupera_lista` abaixo, que dado um tensor que representa o grid de saída da nossa rede neural, retorna uma lista de BBs e uma lista de suas classes associadas ao grid após procedimento de supressão não-maximal. Se tiver dúvidas sobre NMS, veja o vídeo abaixo do Andrew Ng:

[![Video Explaining NMS](https://i.ytimg.com/vi/VAo84c1hQX8/maxresdefault.jpg)](https://www.youtube.com/watch?v=VAo84c1hQX8 "Video Explaining NMS - Click to Watch!")

**Sugiro fortemente implementar a função aos poucos (TDD)**

**NÃO** use LLMs (ChatGPT) para pegar a resposta pronta. **NÃO** pesquise a resposta pronta na internet.

**Pode** olhar a documentação das bibliotecas (PyTorch, FastAI, mas todas as funções que você precisa está nas **dicas**) e **pode** (aconselhado) olhar o material de aula (slides e referências).

<details><summary><b>Dica para a resposta</b></summary>
<p>

Use a função `IoU` da explicação e a forma e percorrer o loop já ordenado pelas probabilidades.

Funções pertinentes da biblioteca: `torch.exp`, `torch.cat`, `torch.sort` (use a ordem correta com o argumento `descending`)

Use `.numpy().tolist()` para converter um tensor para uma lista, e `.item()` para pegar o valor escalar do tensor (de rank 0)

Use `lista4.append(elem76)` para adicionar um elemento na lista de Python.

A minha implementação final (dentro do solution) ficou com 15 linhas. Seja esperto no broadcasting e nos slices.

</p>
</details>

In [ ]:
# questao_nms autograded_answer

def recupera_lista(saida: Tensor, S: int, norms_bb: Tensor, thres=0.3, iou_nms=0.5):
    """Extrai a lista de BBs e realiza a supressão não maximal.
    
    Args:
        saida: tensor de shape (1+4+C, H_g, W_g) que representa o grid de BBs da rede
        S: tamanho do grid em pixels
        norms_bb: largura e altura média (fator de normalização) das BBs
        thres: limiar (threshold) dos valores de probabilidade, se a probabilidade
            for abaixo desse limiar, o candidato deve ser descartado
        iou_nms: valor de IoU para supressão não-maximal (NMS), se o IoU for acima
            desse limiar o candidato deve ser descartado
    
    Returns:
        lista de bouding boxes definidas pelo vértice do topo esquerdo, largura e altura (x, y, w, h)
        lista de classes: cada classe é um inteiro entre 0 e C-1 inclusive
    """
    Ct, Hg, Wg = saida.shape
    H, W = S * Hg, S * Wg
    C = Ct -5
    lista_bbs, lista_classes = [], []
    xy = torch.stack(torch.meshgrid(torch.arange(Hg), torch.arange(Wg), indexing='ij')[::-1])
    # WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
    raise NotImplementedError()

Se você usou uma LLM, escreva a sua conversa com ela aqui nesta própria célula de texto (copie a conversa inteira) ou exporte o link da conversa:

**Escreva aqui**

Para um tensor de entrada com probabilidades nulas, deve retornar uma lista vazia:

In [ ]:
# testa_1_questao_nms autograder_tests 0.2

tensor_teste = torch.zeros(15, 10, 10)

lista_bbs, categorias = recupera_lista(tensor_teste, 100, tensor([1., 1.]))

assert lista_bbs == []
assert categorias == []

Para um tensor com um grid unitário, e probabilidade acima do threshold, deve realizar a desnormalização dos valores:

In [ ]:
# testa_2_questao_nms autograder_tests 0.3

tensor_teste2 = tensor([1, -0.1, 0.2, np.log(1.5), np.log(0.5), 0, 0, 1, 0]).reshape(9, 1, 1)

lista_bbs2, categorias2 = recupera_lista(tensor_teste2, 100, tensor([50., 50.]))

assert len(lista_bbs2) == 1
assert len(categorias2) == 1
assert type(lista_bbs2[0]) == list
assert np.linalg.norm(array(lista_bbs2[0]) - array([
    45-75/2, 60-25/2, 75, 25
])) < 1e-5
assert type(categorias2[0]) == int
assert categorias2[0] == 2


Deve retornar uma lista de BBs para múltiplos elementos do grid:

In [ ]:
# testa_3_questao_nms autograder_tests 0.5

tensor_teste3 = tensor([
    [0.9, 0, 0, np.log(1.1), np.log(1.2), 0.8, 0.1, 0, 0.1],
    [0.7, 0, 0, np.log(1.2), np.log(1.1), 0.1, 0.1, 0, 0.8],
    [0.5, 0, 0, np.log(1.1), np.log(1.2), 0, 0.1, 0.8, 0.1],
    [0.2, 0, 0, np.log(1.1), np.log(1.2), 0, 0.1, 0.8, 0.1],
]).transpose(0, 1).reshape(9, 2, 2)
bbs_esperados3 = [
    [50-55, 50-60, 110, 120],
    [150-60, 50-55, 120, 110],
    [50-55, 150-60, 110, 120]
]

lista_bbs3, categorias3 = recupera_lista(tensor_teste3, 100, tensor([100., 100.]), thres=0.3)

assert len(lista_bbs3) == 3
assert len(categorias3) == 3
assert np.linalg.norm(array(lista_bbs3) - array(bbs_esperados3)) < 1e-3
assert np.all(array(categorias3) == array([0, 3, 2]))


Ainda no mesmo exemplo anterior, vamos deixar as BBs com um overlapping muito grande entre elas, nós devemos fazer NMS, atente que a ordem de retorno da BBs deve ser pela ordem das probabilidades:

In [ ]:
# testa_4_questao_nms autograder_tests 1

tensor_teste4 = tensor([
    [0.7,  0.99,  0.99, np.log(1.9), np.log(1.9), 1, 0],
    [0.6, -0.99,  0.99, np.log(1.9), np.log(1.9), 1, 0],
    [0.9,  0.99, -0.99, np.log(1.9), np.log(1.9), 1, 0],
    [0.5, -0.99, -0.99, np.log(1.9), np.log(1.9), 1, 0],
]).transpose(0, 1).reshape(7, 2, 2)
bbs_esperados4 = [
    [99.5-190/2, 100.5-190/2, 190, 190],
]

lista_bbs4, categorias4 = recupera_lista(tensor_teste4, 100, tensor([100., 100.]), thres=0.3)

assert len(lista_bbs4) == 1
assert len(categorias4) == 1
assert np.linalg.norm(array(lista_bbs4) - array(bbs_esperados4)) < 1e-3
assert categorias4[0] == 0

Finalmente, vamos aplicar em um exemplo maior:

In [ ]:
# testa_5_questao_nms autograder_tests 2

tensor_teste5 = tensor([
[0.7,  0.99,  0.99, np.log(1.9), np.log(2), 1, 0], [0.8, -0.99,  0.99, np.log(1.9), np.log(2), 1, 0],
[0.6,  0.99, -0.99, np.log(1.9), np.log(2), 1, 0], [0.9, -0.91, -0.99, np.log(2), np.log(1.9), 0, 1],
[0.5,     0, -0.99, np.log(1.9), np.log(2), 1, 0], [0.2, -0.99, -0.99, np.log(1.9), np.log(2), 1, 0],
[0.7,  0.99,     0, np.log(2),   np.log(2), 0, 1], [1.0, -0.99,     0, np.log(2),   np.log(2), 0, 1],
]).transpose(0, 1).reshape(7, 4, 2)
bbs_esperados5 = [
    [141, 640, 120, 120],
    [149, 144, 120, 114],
    [144, 139, 114, 120],
    [43,  341, 114, 120]
]

lista_bbs5, categorias5 = recupera_lista(tensor_teste5, 200, tensor([60., 60.]), thres=0.3, iou_nms=0.4)

assert len(lista_bbs5) == 4
assert len(categorias5) == 4
assert np.linalg.norm(array(lista_bbs5) - array(bbs_esperados5)) < 1e-3
assert np.all(array(categorias5) == array([1, 1, 0, 0]))

Veja se a sua resposta está fazendo sentido, lembre-se de que classes diferentes são independentes no NMS (assim pode ter um verde com alto IoU em relação a um azul)

In [ ]:
plot_dado({'image': Image.fromarray(np.zeros((800, 400, 3), dtype=np.uint8)), 
           'objects': {'bbox': lista_bbs5, 'category': categorias5}}, grid=200)
tensor_teste5[0], lista_bbs5, categorias5

## Treinando de Verdade! (Use GPU)

Agora vamos usar o FastAI para treinar a rede a partir do transfer learning da outra que era apenas de classificação. Primeiro vamos definir o nosso DataBlock do FastAI:

Vamos definir a nossa função `get_items` que simplesmente retorna uma lista dos itens do dataset, mas que também marca com uma flag os itens de treinamento (obs: essa implementação acabou ficando bronca por que a memória fica alocada na imagem quando ele acessa o dicionário):

In [ ]:
def get_dict_dataset_items(data: dict):
    train = L(data.get('train'))
    for d in train:
        d['train'] = True
    return (train + L(data.get('validation'))).shuffle()

its = get_dict_dataset_items({'train': [{0:'ok'}, {0:'bom'}, {0:'nice'}, {0:'wut'}, {0:'go'}], 
                              'validation': [{0:'dis'}, {0:'mau'}, {0:'est'}]})
its

E o nosso splitter que vê a flag para retornar os índices de treinamento e de validação:

In [ ]:
def train_val_splitter(data_list):
    train, valid = L(), L()
    for i, datum in enumerate(data_list):
        if datum.get('train'):
            train.append(i)
        else:
            valid.append(i)
    return L(train), L(valid)

train_val_splitter(its)

O FastAI padronizou uma tupla de bounding boxes pelos vértices $(x_0, y_0, x_1, y_1)$, em vez de ser com largura e altura $(x_0, y_0, w, h)$. Assim vamos transformar para o formato dele:

In [ ]:
def get_bb_from_item(data_item):
    bbs = array(data_item['objects']['bbox'])
    bbs[..., 2:] += bbs[..., :2]
    return bbs

get_bb_from_item({'objects': {'bbox': [[10, 12, 5, 6], [8, 9, 2, 3]]}})

E também vamos criar o nosso *transform* final que irá chamar a primeira função `monta_grid_detecta` que vocês implementaram.

In [ ]:
class TransformaBB(ItemTransform):
    def __init__(self, C=4, grid_size=32, norms_bb=tensor([55.5, 84.6]), thres=0.3, iou_nms=0.2):
        self.C = C
        self.grid_size = grid_size
        self.norms_bb = norms_bb
        self.thres = thres
        self.iou_nms = iou_nms

O método `encodes` (esse nome é importante para o FastAI) vai receber uma tupla com a imagem, as bounding boxes e as categorias e deve formar o grid.

In [ ]:
@patch_to(TransformaBB)
def encodes(self: TransformaBB, img_bb_cat):
    if len(img_bb_cat) < 3:
        return img_bb_cat
    img, bbs, cats = img_bb_cat
    H, W = img.shape
    bbs[..., 2:] -= bbs[..., :2]
    saida_esperada = monta_grid_detecta(bbs, cats, self.C, W, H, self.grid_size, self.norms_bb)
    return img, saida_esperada

O método `decodes` (nome é importante) faz o inverso do encodes (ele é usado para visualização ou para inferência). Dessa forma nós temos uma transformação que faz parte do Pipeline do FastAI e que consegue trabalhar nos dois sentidos: codificar um grid a partir de uma lista (*encoding*) e decodificar o grid gerando a lista (*decoding*).

É por isso que era tão importante essas duas funções que vocês implementaram, justamente para poder gerar a entrada esperada para treinar a rede e depois também para poder compreender a saída da rede.

In [ ]:
@patch_to(TransformaBB)
def decodes(self: TransformaBB, img_grid):
    img, grid = img_grid
    bbs, cats = recupera_lista(grid, self.grid_size, self.norms_bb, self.thres, self.iou_nms)
    bbs = TensorBBox(bbs)
    bbs[..., 2:] += bbs[..., :2]
    return img, bbs, TensorMultiCategory(cats)

Então vamos instanciar o nosso DataBlock e o nosso DataLoader, perceba que são três tipos de blocos diferentes: o `ImageBlock` que vocês já conhecem que representa uma imagem, o `BBoxBlock` que representa uma lista de bounding boxes, e o `BBoxLblBlock` que representa uma lista de labels associado a cada bounding box respectivamente.

Então utilizamos as nossas funções que definimos anteriormente, observe com cuidado o código abaixo, atenção para o `item_tfms=TransformaBB()` que chama a transformação que nós criamos para poder montar o nosso grid.

In [ ]:
BBoxBlock_nopad = TransformBlock(type_tfms=TensorBBox.create)

dblock = DataBlock(
    blocks = (ImageBlock, BBoxBlock_nopad, BBoxLblBlock(add_na=False)),
    get_items = get_dict_dataset_items,
    splitter = train_val_splitter,
    n_inp = 1,
    get_x = lambda o: o['image'],
    get_y = [get_bb_from_item, lambda o: o['objects']['category']],
    item_tfms=TransformaBB(),
    batch_tfms=Normalize.from_stats(*imagenet_stats)
)

if RETRAIN:
    data = cs_dataset
else:
    data = {'train': [cs_dataset['train'][i] for i in [0, 2, 5, 10, 11, 50, 100, 103]]}

dloader = dblock.dataloaders(data, bs=32 if RETRAIN else 4)

Só que o FastAI é bem esperto e faz muitas coisas ao mesmo tempo, além da transformação que pedimos, ele colocou mais, associadas aos blocos. Veja as transformações que ele executa, respectivamente, logo depois que ele pega um item (`after_item`), em seguida antes de aglutinar os items em um batch `before_batch`, e depois de ter aglutinado os items em um batch `after_batch`.

Você consegue encontrar o `TransformaBB` abaixo?

In [ ]:
dloader.after_item,  dloader.before_batch, dloader.after_batch

Veja como os dados chegam ao modelo. Perceba o shape dos tensores, é justamente a imagem e o grid, depois de terem passado pela sequência dos Pipelines aí em cima. Perceba também que o FastAI já cuidou para eles ficarem em GPU (se você tiver).

In [ ]:
x, y = next(iter(dloader.train))
x.shape, y.shape, x, y

Vamos ver o batch gerado pelo FastAI. Isso é o resultado primeiro do `encode` e depois do `decode` (ambos tem que estar funcionando e casados no sentido de um fazer o inverso do outro).

In [ ]:
if RETRAIN:
    dloader.show_batch()

Vamos criar o nosso modelo de detecção (que você implementou o `forward`) e carregar os pesos da rede que era apenas de classificação nele.

In [ ]:
model = ConvDect(4)

classification_weights = {}
if RETRAIN:
    url = "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_22k_224.pth"
    classification_weights = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)['model']

transfer_learning(model, classification_weights)

[(nome, param.requires_grad) for nome, param in model.named_parameters()]

Em seguida vamos criar o `Learner` do FastAI, que encapsula tanto o nosso modelo do Pytorch, quanto o dataloader do FastAI, e a loss function, além das métricas e outros detalhes ...

In [ ]:
learner = Learner(dloader, model, nanoyolo_loss(1, 1, 1, 1))

O FastAI implementa o algoritmo de busca de learning rate ([aula 3, slide 29](https://docs.google.com/presentation/d/1pbXPJvpGoDK03KsazRf3D99-FXNd2FLcXpIVPl797PA/edit?pli=1#slide=id.g267c62e9f27_1_303)), que é crucial para treinar grandes modelos, é muito mais inteligente do que ficar treinando várias vezes com learning rates diferentes. Veja abaixo:

**(Use GPU)**

In [ ]:
if RETRAIN:
    learner.lr_find()

Vamos ser um pouco mais audazes e colocar um learning rate maior do que ele sugeriu. Mas tenha cuidado, pois o gráfico do `lr_find` é de uma *loss smoothed*, ou seja, ela tem **atraso**, então tem que ser antes do ponto de inflexão.

Assim, executamos os loops de treinamento por N épocas por meio da função abaixo, com a learning rate desejada:

In [ ]:
if RETRAIN:
    learner.fit_one_cycle(2, 0.003)

Agora que as nossas últimas camadas já estão razoavelmente bem treinadas, podemos *descongelar* as camadas anteriores (que estavam com `require_grad=False`) e deixar a rede treinar por inteiro. Isso potencialmente permite com que a rede se adapte melhor ao nosso problema, treinando por inteiro:

In [ ]:
for m in model.modules():
    m.train()
for p in model.parameters():
    p.requires_grad = True

learner = Learner(dloader, model, nanoyolo_loss(1, 1, 1, 1))

[(nome, param.requires_grad) for nome, param in model.named_parameters()]

E fazemos agora mais uma época de treino, vamos usar um learning rate menor. Percebe que está demorando muito mais por época, você consegue saber porque? Responda na célula abaixo (obs: essa resposta discursiva não vai contabilizar na nota, mas irei ler)

In [ ]:
if RETRAIN:
    learner.fit_one_cycle(1, 0.0003)

WRITE YOUR SOLUTION HERE! (do not change this first line):

**ATTENTION**

**ATTENTION**

**ATTENTION**

**ATTENTION**

**DISCURSIVE QUESTION**

WRITE YOUR ANSWER HERE (do not delete this cell so the ID is not lost)

**ATTENTION**

**ATTENTION**


Vamos ver os resultados no conjunto de validação, pode executar a célula abaixo do `show_results` por várias vezes, cada vez serão imagens da validação aleatórias:

In [ ]:
if RETRAIN:
    learner.show_results() # me rode várias vezes, imagens diferentes!

Vamos testar com outras imagens que eu peguei aleatóriamente da internet! Sinta-se à vontade para trocar o nome da imagem ou até para baixar a sua própria ou tirar um print do seu jogo (*para fins puramente de pesquisa, é claro* 😏).

In [ ]:
img_teste = None
if RETRAIN:
    img_teste = PILImage.create(base_path/'cs02.jpg') # cs00.jpg, ... cs08.jpg
img_teste

Basta executar o método `predict` com a nossa imagem do Pillow, infelizmente ele já não faz o plot automático igual o show_results, mas podemos nós mesmos ver os resultados:

In [ ]:
resultado_predict_teste = None
if RETRAIN:
    resultado_predict_teste = learner.predict(img_teste)
resultado_predict_teste

Vamos usar a nossa própria função para ver os resultados.

In [ ]:
if RETRAIN:
    bbs_teste, cats_teste = resultado_predict_teste[0][1]
    bbs_teste = bbs_teste.detach().clone()
    bbs_teste[:, 2:] -= bbs_teste[:, :2]
    plot_dado({'image': img_teste, 'objects': {'bbox': bbs_teste, 'category': cats_teste}})

In [ ]:
if RETRAIN:
    torch.save(model.state_dict(), base_path/'treinado_meu.pth')

Ainda há pontos de melhoria para o lab, mas já ficou bem extenso então vou deixar para apresentar no próximo ou deixar citado:

É interessante colocar um Augmentation no treino e treinar por mais épocas também. Como nós treinamos por poucas épocas a falta do data augmentation não foi tão grave assim. Mas para se ter um resultado melhor é imprescindível usá-lo. O aug_transforms era pensado para o batch_tfms, mas nós temos que colocar as augmentations no item_tfms antes da nossa criação do grid, e ele deve alterar tanto a imagem quanto as bounding boxes da mesma forma.

Outro ponto importantíssimo que também não foi explorado são as métricas de detecção: o MaP (mean average precision), e também um gráfico de precision vs recall para os diferentes thresholds.

Além disso o nosso modelo é bem simples, na questão de ter apenas uma bounding box por elemento do grid, além de ter uma função custo simplificada que não leva em consideração o IoU. Inclusive deixo essas melhorias como propostas para o trabalho do exame!

# Your data and feedback:

Write a feedback for the lab so we can make it better for the next years.

In the following variables, write the number of hours spent on this lab, the perceived difficulty, and the expected grade (you may delete the `raise` and the comments):

In [ ]:
# meta_eval manual_graded_answer 0

horas_gastas = None    # 1.5   - Float number with the number of hours spent 
dificuldade_lab = None # 0     - Float number from 0.0 to 10.0 (inclusive)
nota_esperada = None   # 10    - Float number from 0.0 to 10.0 (inclusive)

# WRITE YOUR CODE HERE! (you can delete this comment, but do not delete this cell so the ID is not lost)
raise NotImplementedError()

Write below other comments or feedbacks about the lab. If you did not understand anything about the lab, please also comment here.

If you find any typo or bug in the lab, please comment below so we can fix it.

WRITE YOUR SOLUTION HERE! (do not change this first line):

**ATTENTION**

**ATTENTION**

**ATTENTION**

**ATTENTION**

**DISCURSIVE QUESTION**

WRITE YOUR ANSWER HERE (do not delete this cell so the ID is not lost)

**ATTENTION**

**ATTENTION**


**End of the lab!**